In [0]:
CREATE OR REPLACE TEMPORARY FUNCTION clean_date(d DATE)
RETURNS DATE
RETURN CASE
  WHEN d IS NULL THEN NULL
  WHEN d < DATE '1901-01-01' THEN NULL
  WHEN d IN (
    DATE '1900-01-01', DATE '1901-01-01',
    DATE '1969-12-31', DATE '1970-01-01',
    DATE '2099-12-31', DATE '2999-12-31', DATE '9999-12-31'
  ) THEN NULL
  WHEN d > current_date() + INTERVAL 3 YEARS THEN NULL
  ELSE d
END;

--CREATE OR REPLACE TABLE teamconnect.pca_prism.07_tbl_trt_latest AS
INSERT OVERWRITE TABLE teamconnect.pca_prism.07_tbl_trt_latest
SELECT
  btn,
  dpi_environment,
  remdte,
  remamt,
  spsdte,
  spsamt,
  opnsps,
  nsfdte,
  opnnsf,
  dpadte,
  opndpa,
  ovrdte,
  opnovr,
  pyxdte,
  treatment_type,
  CASE
    WHEN LOWER(TRIM(CAST(sedqupsl AS STRING))) IN ('y','yes','1','true','t')
      THEN 'Y'
    ELSE 'N'
  END AS disqualified_upsell_sne_yn
FROM (
  SELECT
    *,
    ROW_NUMBER() OVER (
      PARTITION BY btn, dpi_environment
      ORDER BY GREATEST(
        COALESCE(remdte, DATE '1900-01-01'),
        COALESCE(spsdte, DATE '1900-01-01'),
        COALESCE(nsfdte, DATE '1900-01-01'),
        COALESCE(dpadte, DATE '1900-01-01'),
        COALESCE(ovrdte, DATE '1900-01-01'),
        COALESCE(pyxdte, DATE '1900-01-01')
      ) DESC
    ) AS rn
  FROM teamconnect.pca_prism.06_tbl_trt_raw_clean
  WHERE btn IS NOT NULL
    AND btn RLIKE '^[0-9]{10}$'
) x
WHERE rn = 1;


OPTIMIZE teamconnect.pca_prism.07_tbl_trt_latest
ZORDER BY (btn, dpi_environment);
